In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_table(PATH["w.tsv"], index_col=0)

w.columns.name = "Factor"

h = pd.read_table(PATH["h.tsv"], index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, df, normalization_axis, cluster_axis in (("w", w, 1, 0), ("h", h, 0, 1)):

    ccal.plot_heat_map(
        df,
        normalization_axis=normalization_axis,
        normalization_method="-0-",
        cluster_axis=cluster_axis,
        title="NMF {}".format(w_or_h.title()),
        xaxis_title=df.columns.name,
        yaxis_title=df.index.name,
        html_file_path="{}/heat_map.html".format(PATH["{}/".format(w_or_h)]),
    )

In [ ]:
for element_type, w_or_h in (("feature", "w"), ("sample", "h")):

    if SETTING["{}s_to_peek".format(element_type)] is None:

        continue

    if element_type == "feature":

        df = w.loc[
            [
                feature
                for feature in w.index
                if any(
                    feature_to_peek == feature.replace("(-) ", "").replace("(+) ", "")
                    for feature_to_peek in SETTING["{}s_to_peek".format(element_type)]
                )
            ]
        ]

    elif element_type == "sample":

        df = h[
            [
                sample
                for sample in h.columns
                if any(
                    sample_to_peek == sample.replace("(-) ", "").replace("(+) ", "")
                    for sample_to_peek in SETTING["{}s_to_peek".format(element_type)]
                )
            ]
        ]

    ccal.plot_bubble_map(
        df,
        title="{} to Peek".format(SETTING["{}_alias".format(element_type)]),
        xaxis_title=df.columns.name,
        yaxis_title=df.index.name,
        html_file_path="{}/bubble_map.html".format(PATH["{}/".format(w_or_h)]),
    )

In [ ]:
signal = pd.read_table(PATH["feature_x_sample.signal.tsv"], index_col=0)

signal.columns.name = SETTING["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH["feature_x_sample.processed.tsv"], index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, targets, data_signal, data_raw in (
    ("w", w.T, signal.T, feature_x_sample_processed.T),
    ("h", h, signal, feature_x_sample_processed),
):

    ccal.make_match_panels(
        targets,
        {
            "Signal": {"df": data_signal, "type": "continuous"},
            "Feature-x-Sample": {"df": data_raw, "type": "continuous"},
        },
        directory_path=PATH["{}|match/".format(w_or_h)],
        n_required_for_match_function=SETTING["n_required_for_match_function"],
        n_extreme=SETTING["n_extreme"],
        plot_std=SETTING["plot_std"],
    )